In [61]:
import ipywidgets as widgets
from IPython.display import display

S_30 = 30
M_1  = 60
M_2  = 120
M_5  = 300
M_15 = 900

In [110]:
time_frame = S_30
print("Selected time frame " + str(time_frame))

Selected time frame 30


In [111]:
#load data
import csv
import os
from datetime import datetime, timedelta
import time

raw_candles = {}

#should be in input

BID_COLUMN = 2
ASK_COLUMN = 1

with open(os.path.join(os.getcwd(), '../data/EURUSD_Ticks_08.02.2016-08.02.2016.csv'), 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    reader.__next__()
    reader.__next__()
    for row in reader:
        ts = time.mktime(datetime.strptime(row[0], '%d.%m.%Y %H:%M:%S.%f').timetuple())
        key = int(ts - (ts%time_frame))
        if raw_candles.get(key) is None:
            raw_candles[key] = []
        raw_candles[key].append(row[BID_COLUMN])

In [112]:
#Trasform data in a candle

class Candle:
    
    def __init__(self, timestamp, timeframe, sortedValues):
        self.timestamp = timestamp
        self.timeframe = timeframe
        self.open_price = sortedValues[0]
        self.close_price = sortedValues[len(sortedValues) - 1]
        self.high  = max(sortedValues)
        self.low = min(sortedValues)
        
    def __str__(self):
        return str(datetime.fromtimestamp(self.timestamp)) \
         + " { H:" + str(self.high) + " L:" + str(self.low) + " O: " \
         + str(self.open_price) + " C: " + str(self.close_price)+ " }"
     
    def __repr__(self):
        return self.__str__()

candles = []

for k in list(raw_candles.keys()):
    candles.append(Candle(k, time_frame, raw_candles[k]))

candles.sort(key=lambda x: x.timestamp)

for c in candles[0:10]:
    print(c)

2016-02-08 00:00:00 { H:1.11441 L:1.11419 O: 1.11434 C: 1.11434 }
2016-02-08 00:00:30 { H:1.11441 L:1.11433 O: 1.11434 C: 1.11436 }
2016-02-08 00:01:00 { H:1.11437 L:1.11425 O: 1.11437 C: 1.11427 }
2016-02-08 00:01:30 { H:1.11431 L:1.11425 O: 1.11427 C: 1.11425 }
2016-02-08 00:02:00 { H:1.11428 L:1.11417 O: 1.11428 C: 1.11417 }
2016-02-08 00:02:30 { H:1.11425 L:1.11417 O: 1.11417 C: 1.11425 }
2016-02-08 00:03:00 { H:1.11430 L:1.11425 O: 1.11425 C: 1.11430 }
2016-02-08 00:03:30 { H:1.11431 L:1.11430 O: 1.11430 C: 1.11430 }
2016-02-08 00:04:00 { H:1.11430 L:1.11430 O: 1.11430 C: 1.11430 }
2016-02-08 00:04:30 { H:1.11430 L:1.11430 O: 1.11430 C: 1.11430 }


In [113]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode()
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
from datetime import datetime

In [114]:
from pandas import DataFrame, Series

index = [ datetime.fromtimestamp(x.timestamp) for x in candles]

d = {
    'Open': Series([ x.open_price for x in candles], index=index), 
    'Close': Series([ x.close_price for x in candles], index=index),
    'Low': Series([ x.low for x in candles], index=index),
    'High': Series([ x.high for x in candles], index=index)
}

df = DataFrame(d)

In [115]:
#validation

for (x,y) in zip(df.High, df.Low):
    if x < y:
        print("Data invalid")

In [116]:
fig = FF.create_candlestick(df.Open, df.High, df.Low, df.Close, dates=df.index)

In [117]:
iplot(fig)